In [9]:
from typing import List, Dict
import wikipediaapi as wk


In [52]:
class DataExtractor():
    
    def __init__(self, start:int=1900, stop:int=2000, drange:list=None)->None:
        self.start = start
        self.stop = stop
        self.drange = drange

    

In [56]:
def generate_drange(start, stop)->List[str]:
        """
        Returns list of decades
        """
        return [f"{str(i)}s" for i in range(start, stop+1, 10)]